In [ ]:
%matplotlib inline

import nltk
nltk.download('stopwords')
import nltk
import numpy as np
import pandas as pd
import re
import string
import os
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data =pd.read_csv(r'/content/cleaned_tweets (3).csv')
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,final_text,lemmatized,tokens
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),what said,what said,"['what', 'said']"
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),plus you have added commercial experience tacky,plus you have added commercial experience tacky,"['plus', 'you', 'have', 'added', 'commercial',..."
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),i did not today must mean i need take another ...,i did not today must mean i need take another ...,"['i', 'did', 'not', 'today', 'must', 'mean', '..."
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),bad flight it is really aggressive blast obnox...,bad flight it is really aggressive blast obnox...,"['bad', 'flight', 'it', 'is', 'really', 'aggre..."
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),ca not tell it is really big bad thing it,ca not tell it is really big bad thing it,"['ca', 'not', 'tell', 'it', 'is', 'really', 'b..."


In [ ]:
# Separate negative and positive sentiment tweets
neg_data = data[data['airline_sentiment'] == 'negative']
pos_data = data[data['airline_sentiment'] == 'positive']

# Sample an equal number of negative and positive tweets
num_samples = min(len(neg_data), len(pos_data))
neg_data = neg_data.sample(n=num_samples, random_state=42)
pos_data = pos_data.sample(n=num_samples, random_state=42)

# Concatenate the negative and positive sentiment tweets
balanced_data = pd.concat([neg_data, pos_data])

# Shuffle the rows
balanced_data = balanced_data.sample(frac=1, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

# Split into features and target
X = balanced_data['final_text'].values.tolist()
y = balanced_data['airline_sentiment'].values.tolist()

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state = 42)
# Map positive to 1 and negative to 0
y_train = [int(sent == 'positive') for sent in y_train]
y_test = [int(sent == 'positive') for sent in y_test]
def build_freqs(tweets, ys):
    yslist = np.squeeze(ys).tolist()
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1
    return freqs
    # Calculate the number of tweets, the number of positive tweets, the number of negative tweets
T = len(X_train)
T_pos = y_train.count(1)
T_neg = y_train.count(0)

print('Total number of tweets: {}'.format(T))
print('Number of positive tweets: {}'.format(T_pos))
print('Number of negative tweets: {}'.format(T_neg))
# Calculate the probabilities for each class
Prob_T_pos = T_pos/T
Prob_T_neg = T_neg/T

print('Probability of positive class: {}'.format(Prob_T_pos))
print('Probability of negative class: {}'.format(Prob_T_neg))
# Calculate the number of unique words V
vocab = [key[0] for key in freqs.keys()]
V = len(set(vocab))

print('Number of unique words: {}'.format(V))
# Calculate N_pos, N_neg
N_pos = 0
N_neg = 0

for pair in freqs.keys():
    # if label is positive
    if pair[1] > 0:
        N_pos += freqs[pair]
    # if label is negative
    else:
        N_neg += freqs[pair]

print("Number of positive words for all tweets: {}".format(N_pos))
print("Number of negative words for all tweets: {}".format(N_neg))

# Calculation for positive and negative probability of a word. We will do it for the first word in the vocab
print('First Word in vocab: {}'.format(vocab[0]))

# get positive and negative frequency of word
freq_pos = freqs.get((vocab[0],1), 0)
freq_neg = freqs.get((vocab[0],0), 0)

print('Positive frequency of {} is: {}'.format(vocab[0], freq_pos))
print('Negative frequency of {} is: {}'.format(vocab[0], freq_neg))

# calculate the probability that the word is positive and negative
p_w_pos = (freq_pos + 1)/(N_pos + V)
p_w_neg = (freq_neg + 1)/(N_neg + V)

print("The probability that the word: {} is positive is: {}".format(vocab[0], p_w_pos))
print("The probability that the word: {} is negative is: {}".format(vocab[0], p_w_neg))
# calculate the log likelihood of the word
loglikelihood = np.log(p_w_pos/ p_w_neg)

print("The log likelihood of the word: {} is: {}".format(vocab[0], loglikelihood))
def train_naive(freqs, X_train, y_train):

    loglikelihood = {}
    logprior = 0

    # Calculate the number of unique words V
    vocab = [key[0] for key in freqs.keys()]
    V = len(set(vocab))

    # Calculate N_pos, N_neg
    N_pos = 0
    N_neg = 0

    for pair in freqs.keys():
        # if label is positive
        if pair[1] > 0:
            N_pos += freqs[pair]
        # if label is negative
        else:
            N_neg += freqs[pair]

    # Calculate the number of tweets, the number of positive tweets, the number of negative tweets
    T = len(X_train)
    T_pos = y_train.count(1)
    T_neg = y_train.count(0)

    # Calculate the logprior
    logprior = np.log(T_pos) - np.log(T_neg)

    # for each word in the vocabulary
    for word in vocab:
        # get positive and negative frequency of word
        freq_pos = freqs.get((word,1), 0)
        freq_neg = freqs.get((word,0), 0)

        # calculate the probability that the word is positive and negative
        p_w_pos = (freq_pos + 1)/(N_pos + V)
        p_w_neg = (freq_neg + 1)/(N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos/ p_w_neg)

    return logprior, loglikelihood

    logprior, loglikelihood = train_naive(freqs, X_train, y_train)
    # Print the first five key-value pairs of loglikelihood
count = 0
for key, value in loglikelihood.items():
    if count < 5:
        print(key, value)
        count += 1
    else:
        break
        def predict_naive(tweet, logprior, loglikelihood):

    # process tweet to return list of cleaned words
    words = process_tweet(tweet)

    # intialize the probability to 0
    p = 0

    # add the logprior
    p += logprior

    # iterate over each word in tweet
    for word in words:
        # check if word in loglikelihood dictionary
        if word in loglikelihood:
            # add the loglikelihood value of the word to the probability
            p += loglikelihood[word]

    return p
    # test on some random string
random_test = 'I am very happy today :)'
predict_naive(random_test, logprior, loglikelihood)
# initialize prediction list
y_preds = []

for tweet in X_test:
    # if prediction is positive, append 1 to y_preds
    if predict_naive(tweet, logprior, loglikelihood) > 0:
        y_preds.append(1)
    # if prediction is negative, append 0 to y_preds
    else:
        y_preds.append(0)

In [ ]:
# Map positive to 1 and negative to 0
y_train = [int(sent == 'positive') for sent in y_train]
y_test = [int(sent == 'positive') for sent in y_test]

In [ ]:
def build_freqs(tweets, ys):
    yslist = np.squeeze(ys).tolist()
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1
    return freqs

In [ ]:
# Calculate the number of tweets, the number of positive tweets, the number of negative tweets
T = len(X_train)
T_pos = y_train.count(1)
T_neg = y_train.count(0)

print('Total number of tweets: {}'.format(T))
print('Number of positive tweets: {}'.format(T_pos))
print('Number of negative tweets: {}'.format(T_neg))

Total number of tweets: 3780
Number of positive tweets: 1877
Number of negative tweets: 1903


In [ ]:
# Calculate the probabilities for each class
Prob_T_pos = T_pos/T
Prob_T_neg = T_neg/T

print('Probability of positive class: {}'.format(Prob_T_pos))
print('Probability of negative class: {}'.format(Prob_T_neg))

Probability of positive class: 0.49656084656084654
Probability of negative class: 0.5034391534391535


In [ ]:
# Calculate the number of unique words V
vocab = [key[0] for key in freqs.keys()]
V = len(set(vocab))

print('Number of unique words: {}'.format(V))

Number of unique words: 4617


In [ ]:
# Calculate N_pos, N_neg
N_pos = 0
N_neg = 0

for pair in freqs.keys():
    # if label is positive
    if pair[1] > 0:
        N_pos += freqs[pair]
    # if label is negative
    else:
        N_neg += freqs[pair]

print("Number of positive words for all tweets: {}".format(N_pos))
print("Number of negative words for all tweets: {}".format(N_neg))

Number of positive words for all tweets: 14235
Number of negative words for all tweets: 24460


In [ ]:
# Calculation for positive and negative probability of a word. We will do it for the first word in the vocab
print('First Word in vocab: {}'.format(vocab[0]))

# get positive and negative frequency of word
freq_pos = freqs.get((vocab[0],1), 0)
freq_neg = freqs.get((vocab[0],0), 0)

print('Positive frequency of {} is: {}'.format(vocab[0], freq_pos))
print('Negative frequency of {} is: {}'.format(vocab[0], freq_neg))

# calculate the probability that the word is positive and negative
p_w_pos = (freq_pos + 1)/(N_pos + V)
p_w_neg = (freq_neg + 1)/(N_neg + V)

print("The probability that the word: {} is positive is: {}".format(vocab[0], p_w_pos))
print("The probability that the word: {} is negative is: {}".format(vocab[0], p_w_neg))

First Word in vocab: anoth
Positive frequency of anoth is: 24
Negative frequency of anoth is: 53
The probability that the word: anoth is positive is: 0.0013261192446424783
The probability that the word: anoth is negative is: 0.001857137944079513


In [ ]:
# calculate the log likelihood of the word
loglikelihood = np.log(p_w_pos/ p_w_neg)

print("The log likelihood of the word: {} is: {}".format(vocab[0], loglikelihood))

The log likelihood of the word: anoth is: -0.33677974710036085


In [ ]:
def train_naive(freqs, X_train, y_train):
    """
    Inputs:
        freqs: dictionary {(word, label): count}
        X_train: list of tweets
        y_train: list of labels
    Outputs:
        logprior: the log prior value
        loglikelihood: log likelihood of the Naive Bayes Equation.
    """

    loglikelihood = {}
    logprior = 0

    # Calculate the number of unique words V
    vocab = [key[0] for key in freqs.keys()]
    V = len(set(vocab))

    # Calculate N_pos, N_neg
    N_pos = 0
    N_neg = 0

    for pair in freqs.keys():
        # if label is positive
        if pair[1] > 0:
            N_pos += freqs[pair]
        # if label is negative
        else:
            N_neg += freqs[pair]

    # Calculate the number of tweets, the number of positive tweets, the number of negative tweets
    T = len(X_train)
    T_pos = y_train.count(1)
    T_neg = y_train.count(0)

    # Calculate the logprior
    logprior = np.log(T_pos) - np.log(T_neg)

    # for each word in the vocabulary
    for word in vocab:
        # get positive and negative frequency of word
        freq_pos = freqs.get((word,1), 0)
        freq_neg = freqs.get((word,0), 0)

        # calculate the probability that the word is positive and negative
        p_w_pos = (freq_pos + 1)/(N_pos + V)
        p_w_neg = (freq_neg + 1)/(N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos/ p_w_neg)

    return logprior, loglikelihood

In [ ]:
logprior, loglikelihood = train_naive(freqs, X_train, y_train)

In [ ]:
print("Calculate Logprior: {}".format(logprior))


Calculate Logprior: -0.01375683070964051


In [ ]:
# Print the first five key-value pairs of loglikelihood
count = 0
for key, value in loglikelihood.items():
    if count < 5:
        print(key, value)
        count += 1
    else:
        break

anoth -0.33677974710036085
awesom 4.206089412690352
telephon 0.4333284745957127
experi 0.3155454389393292
thank 2.8727727503069556


In [ ]:
def predict_naive(tweet, logprior, loglikelihood):
    """
    Predict the sentiment of tweet

    Inputs:
        tweet: string
        logprior: number
        loglikelihood: dictionary {'word': loglikelihood_value}
    Outputs:
        p: sum of all the loglikelihood of the words that appear in the tweet plus the logprior value.
    """
    # process tweet to return list of cleaned words
    words = process_tweet(tweet)

    # intialize the probability to 0
    p = 0

    # add the logprior
    p += logprior

    # iterate over each word in tweet
    for word in words:
        # check if word in loglikelihood dictionary
        if word in loglikelihood:
            # add the loglikelihood value of the word to the probability
            p += loglikelihood[word]

    return p

In [ ]:
# test on some random string
random_test = 'I am very happy today :)'
predict_naive(random_test, logprior, loglikelihood)

np.float64(1.8747427352613122)

In [ ]:
# initialize prediction list
y_preds = []

for tweet in X_test:
    # if prediction is positive, append 1 to y_preds
    if predict_naive(tweet, logprior, loglikelihood) > 0:
        y_preds.append(1)
    # if prediction is negative, append 0 to y_preds
    else:
        y_preds.append(0)

In [ ]:
# calculate the accuracy
from sklearn.metrics import accuracy_score

y_test = y_test

accuracy = accuracy_score(y_test, y_preds)

print("The accuracy of the Naives Bayes Model is: {}".format(accuracy))

The accuracy of the Naives Bayes Model is: 0.9312896405919662


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       460
           1       1.00      0.87      0.93       486

    accuracy                           0.93       946
   macro avg       0.94      0.93      0.93       946
weighted avg       0.94      0.93      0.93       946

